In [8]:
import loadAndClean
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation

In [3]:
data = loadAndClean.loadAndClean()
data.describe()

,Provider Id,Provider Zip Code,Total Discharges,Average Covered Charges Num,Average Total Payments Num,Average Medicare Payments Num,DRG Code,Latitude,Longitude
count,153600.000000,153600.000000,153600.000000,153600.000000,153600.000000,153600.000000,153600.000000,153600.000000,153600.000000
mean,255800.984069,48052.483874,42.429115,40864.468878,10424.769029,8848.750017,402.630436,37.527665,-89.767584
std,152328.951075,27906.442510,53.034475,39014.451261,8170.682544,7530.985553,234.099932,5.069681,14.777029
min,10001.000000,1040.000000,11.000000,2398.640000,2233.910000,1220.000000,39.000000,19.519948,-161.755833
25%,110083.000000,27330.000000,17.000000,18066.970000,5607.747500,4386.857500,208.000000,33.902935,-95.695629
50%,250001.000000,44512.000000,27.000000,28706.500000,7737.780000,6393.740000,314.000000,38.576702,-85.828735
75%,380071.000000,72956.000000,48.000000,49047.665000,12129.882500,10483.682500,602.000000,41.335133,-79.942215
max,670085.000000,99801.000000,3680.000000,857124.820000,170123.380000,124829.690000,948.000000,64.831157,-67.998587


In [17]:
clf = RandomForestRegressor(n_estimators=500, n_jobs=4, verbose=3)
predictorsRan = ['Latitude','Longitude','DRG Code']
scores = cross_validation.cross_val_score(clf, data[predictorsRan], data["Average Medicare Payments Num"], scoring='mean_squared_error' ,cv=3)
print scores.mean()

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    3.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   14.5s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   36.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:   40.2s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    5.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   17.2s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   41.5s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:   45.6s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    1.

-71182570.1756


In [18]:
scores

array([-58611787.89855418, -55504780.32192636, -99431142.30621368])